# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [22]:

import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/pamwa/scaling_to_production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [26]:

from sklearn.model_selection import train_test_split


# Define the file paths
data_file_path = 'C:/Users/pamwa/scaling_to_production/05_src/data/adult/adult.data'
test_file_path = 'C:/Users/pamwa/scaling_to_production/05_src/data/adult/adult.test'

# Load the training data
train_data = (pd.read_csv(data_file_path, header=None, names=columns)
              .assign(income=lambda x: (x.income.str.strip() == '>50K')*1))

# Load the test data
test_data = (pd.read_csv(test_file_path, header=None, names=columns, skiprows=1)
             .assign(income=lambda x: (x.income.str.strip() == '>50K.')*1))

# Combine train and test data for a complete dataset
data = pd.concat([train_data, test_data])

# Separate features (X) and target (Y)
X = data.drop('income', axis=1)
Y = data['income']

# Perform train-test split with a 70-30% ratio
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

Random state of the splitting function is a parameter that controls the randomness of the data shuffling and splitting process
The random state ensures that the data is split in a specific, repeatable way.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [29]:

from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify numerical and categorical columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

# Define preprocessing steps for numerical features
num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')), 
    ('scaler', RobustScaler()) 
])

# Define preprocessing steps for categorical features
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  
])

# Combine preprocessing steps into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transformer, num_cols), 
        ('cat_transforms', cat_transformer, cat_cols) 
    ]
)

# Fit and transform the data
X_preprocessed = preprocessor.fit_transform(X)



In [31]:
preprocessor

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Create the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [39]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the scoring metrics
scoring = {
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True),
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}


# Evaluate the model using cross-validation
cv_results = cross_validate(model_pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)

# Report the training and validation results

print(f"Negative Log Loss: {cv_results['train_neg_log_loss'].mean()}")
print(f"ROC AUC: {cv_results['train_roc_auc'].mean()}")
print(f"Accuracy: {cv_results['train_accuracy'].mean()}")
print(f"Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean()}")

print(f"Negative Log Loss: {cv_results['test_neg_log_loss'].mean()}")
print(f"ROC AUC: {cv_results['test_roc_auc'].mean()}")
print(f"Accuracy: {cv_results['test_accuracy'].mean()}")
print(f"Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean()}")




c:\Users\pamwa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\pamwa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\pamwa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Negative Log Loss: -0.07996671152117361
ROC AUC: 0.9999999042842175
Accuracy: 0.999877154855243
Balanced Accuracy: 0.9997799620121366
Negative Log Loss: -0.3787102640711989
ROC AUC: 0.9030054507089774
Accuracy: 0.8530158977876248
Balanced Accuracy: 0.7727465388276838


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [42]:
# Convert the results to a pandas DataFrame
results_df = pd.DataFrame(cv_results)

# Sort the DataFrame by the negative log loss of the test set
results_df_sorted = results_df.sort_values(by='test_neg_log_loss')

# Display the sorted DataFrame
print(results_df_sorted)

    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
3  30.700738    0.448336          -0.389927           -0.079499      0.903814   
0  30.153707    0.431629          -0.385393           -0.079556      0.902078   
4  30.767100    0.447860          -0.377626           -0.080058      0.899126   
2  30.718624    0.453370          -0.372697           -0.080704      0.906811   
1  30.666646    0.442636          -0.367909           -0.080017      0.903198   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
3            1.0       0.853706        0.999898                0.775078   
0            1.0       0.853209        0.999821                0.770132   
4            1.0       0.851044        0.999872                0.768782   
2            1.0       0.856777        0.999872                0.777390   
1            1.0       0.850343        0.999923                0.772352   

   train_balanced_accuracy  
3                 0.999786  
0   

Calculate the mean of each metric. 

In [43]:
# Calculate mean of each metric
print(mean_results)


fit_time                   9.024617
score_time                 0.177633
test_neg_log_loss         -0.375707
train_neg_log_loss        -0.081584
test_roc_auc               0.903373
train_roc_auc              1.000000
test_accuracy              0.854599
train_accuracy             1.000000
test_balanced_accuracy     0.777020
train_balanced_accuracy    1.000000
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Fit the pipeline to the training data
model_pipeline.fit(X_train, Y_train)

# Make predictions and calculate probabilities on the testing data
Y_pred_proba = model_pipeline.predict_proba(X_test)
Y_pred = model_pipeline.predict(X_test)

# Calculate the performance metrics using the predictions and probabilities
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display the results as a dictionary
print(test_metrics)


c:\Users\pamwa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\pamwa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'neg_log_loss': -0.40180313664436956, 'roc_auc': 0.899254402354506, 'accuracy': 0.8545690302327168, 'balanced_accuracy': 0.7764031415969035}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

By using a single step to load and recode the data, the code becomes more concise and easier to read. This can make the script more maintainable and understandable for others who might work with the code.

(Answer here.)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.